In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../dags"))

In [ ]:
import mlflow
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from movie_recommender.recommender import Recommender, Request
from movie_recommender.utils import Timer


Recommender.get_instance().get_model("xgb_cpu");
Recommender.get_instance().get_model("xgb_cuda");
Recommender.get_instance().get_model("dlrm_cpu");
Recommender.get_instance().get_model("dlrm_cuda");

In [ ]:
def batched_vs_unbatched(nrequests, model, ax):
    timers = {}

    def req(): return Request(
        userId=0,
        genres=[],
        model=model,
        temp=0,
        start=0,
        count=10,
    )
    with Timer("batched", mute=True) as t:
        Recommender.batched([req() for i in range(nrequests)])
        timers["batched"] = {} | t
    with Timer("non batched", mute=True) as t:
        for i in range(nrequests):
            Recommender.batched([req()])
        timers["unbatched"] = {} | t

    df = pd.DataFrame([timers["batched"].values(), timers["unbatched"].values()],
                      # type: ignore
                      columns=list(timers["batched"].keys()), index=timers.keys()
                      ).reset_index(names="name")  # type: ignore
    df2 = df.melt(value_vars=["simsearch", "Batch prepare",
                              "Prediction"], id_vars="name", value_name="ms")
    sns.barplot(data=df2, x="variable", y="ms", hue="name", ax=ax)
    ax.set_title(model)

In [ ]:
_, axis = plt.subplots(1, 4, figsize=(16, 4), sharey=True)
batched_vs_unbatched(200, "xgb_cpu", ax=axis[0])
batched_vs_unbatched(200, "dlrm_cpu", ax=axis[1])
batched_vs_unbatched(200, "xgb_cuda", ax=axis[2])
batched_vs_unbatched(200, "dlrm_cuda", ax=axis[3])
plt.tight_layout()